In [ ]:
import sys

sys.path.insert(1, "../")
from utils import *

%matplotlib inline

##### Landsat 

In [ ]:
serie_dirs = [d for d in glob.glob("../data/inputs/spatial/**") if os.path.isdir(d)]

In [ ]:
# dir_index = 0
# dir_path = serie_dirs[dir_index]
# tcds_dir = os.path.join(dir_path, "true_color_ds")
# tc_files = glob.glob(f"{tcds_dir}/**")
# tc_imgs = [flip_img(rasterio.open(f).read()) for f in tc_files]
# plt.figure(figsize=(10, 10), dpi = 300)
# _, axes = plt.subplots(2, 5)
# for i, ax in enumerate(axes.ravel()):
#     ax.imshow(tc_imgs[i])
# plt.tight_layout()

In [ ]:
shifts_dict = {}
for idx in range(len(serie_dirs)):
    print(f"Now processing {os.path.basename(serie_dirs[idx])}")
    dir_path = serie_dirs[idx]
    tcds_dir = os.path.join(dir_path, "true_color")
    tc_files = glob.glob(f"{tcds_dir}/**")
    ref_image = tc_files[1]
    tgt_images = [tc_files[0]] + tc_files[2:]
    output_path = os.path.join(
        f"../data/outputs/OpenCV/spatial/{os.path.basename(serie_dirs[idx])}"
    )
    os.makedirs(output_path, exist_ok=True)
    _, shifts = co_register(
        ref_image,
        tgt_images,
        output_path=output_path,
        remove_outlilers=True,
        # laplacian_kernel_size = 5,
        return_shifted_images=True,
        rethrow_error=True,
        use_overlap=True,
    )
    shifts_dict[os.path.basename(serie_dirs[idx])] = shifts

In [ ]:
raw_images_list = []
ref_images = []
for idx in range(len(serie_dirs)):
    dir_path = serie_dirs[idx]
    tcds_dir = os.path.join(dir_path, "true_color")
    tc_files = glob.glob(f"{tcds_dir}/**")
    raw_images_list.extend(tc_files)
    ref_images.append(tc_files[1])

crg_images_list = []
for ref_image in ref_images:
    coreg_dir = os.path.join("../data/outputs/OpenCV/spatial", ref_image.split("/")[4])
    crg_images_list.extend([ref_image] + glob.glob(f"{coreg_dir}/Aligned/**"))

In [ ]:
dates = set([os.path.basename(im).split("_")[3][0:6] for im in raw_images_list])
concurrent_dict = {}
for date in dates:
    c_raw = list(filter(lambda el: date in el, raw_images_list))
    c_crg = list(filter(lambda el: date in el, crg_images_list))
    cnd = (len(c_raw) == len(c_crg)) and (len(c_raw) > 1)
    if cnd:
        concurrent_dict[date] = {"raw": c_raw, "crg": c_crg}

In [ ]:
raw_mosaics_dir = "../data/outputs/OpenCV/spatial/mosaics/raw"
crg_mosaics_dir = "../data/outputs/OpenCV/spatial/mosaics/crg"
os.makedirs(raw_mosaics_dir, exist_ok=True)
os.makedirs(crg_mosaics_dir, exist_ok=True)
for date in concurrent_dict:
    print(
        f"Now mosaicing scenes for date {date} with {len(concurrent_dict[date]["raw"])} scenes."
    )
    raw_file = os.path.join(raw_mosaics_dir, f"{date}.jpg")
    mosaic, _, _ = make_mosaic(
        concurrent_dict[date]["raw"],
        mosaic_output_path=raw_file.replace(".jpg", ".tif"),
    )
    plt.imsave(raw_file, mosaic, dpi=300)

    crg_file = os.path.join(crg_mosaics_dir, f"{date}.jpg")
    mosaic, _, _ = make_mosaic(
        concurrent_dict[date]["crg"],
        mosaic_output_path=crg_file.replace(".jpg", ".tif"),
    )
    plt.imsave(crg_file, mosaic, dpi=300)

In [ ]:
raw_mosaics = [
    f
    for f in glob.glob("../data/outputs/OpenCV/spatial/mosaics/raw/**")
    if f.endswith(".tif")
]
crg_mosaics = [
    f
    for f in glob.glob("../data/outputs/OpenCV/spatial/mosaics/crg/**")
    if f.endswith(".tif")
]

os.makedirs("../data/outputs/OpenCV/spatial/mosaics_ds/raw", exist_ok=True)
os.makedirs("../data/outputs/OpenCV/spatial/mosaics_ds/crg", exist_ok=True)
for m in raw_mosaics:
    downsample_dataset(m, 0.1, m.replace("mosaics", "mosaics_ds"))
raw_mosaics_ds = glob.glob("../data/outputs/OpenCV/spatial/mosaics_ds/raw/**")

for m in crg_mosaics:
    downsample_dataset(m, 0.1, m.replace("mosaics", "mosaics_ds"))
crg_mosaics_ds = glob.glob("../data/outputs/OpenCV/spatial/mosaics_ds/crg/**")

make_difference_gif(
    raw_mosaics_ds, "../data/outputs/OpenCV/spatial/raw.gif", mosaic_scenes=True
)
make_difference_gif(
    crg_mosaics_ds, "../data/outputs/OpenCV/spatial/crg.gif", mosaic_scenes=True
)